In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import NLocal, CCXGate, CRZGate, RXGate, XGate

from qiskit_algorithms import optimizers
from qiskit.primitives import Estimator

from scipy.optimize import minimize

# import matplotlib.pyplot as plt

from qiskit import *
from qiskit.visualization import *
# from qiskit_aer import Aer

from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

# Para cargar lo necesario
import pickle

In [2]:
def expand_hamiltonian(op, total_qubits):
    expanded_paulis = []
    for pauli, coeff in zip(op.paulis, op.coeffs):
        pauli_str = pauli.to_label()
        # Añadir identidades antes y después según la posición deseada
        new_pauli = (
            pauli_str + "I" * (total_qubits - len(pauli_str))
        )
        expanded_paulis.append((new_pauli, coeff))
    return SparsePauliOp.from_list(expanded_paulis)

In [5]:
# Cargar el ansatz desde el archivo
with open("hamiltonian_hidrogen_reduced.pkl", "rb") as f:
    hamiltonian= pickle.load(f)
print(hamiltonian)

SparsePauliOp(['II', 'ZI', 'IZ', 'ZZ', 'YY', 'XX'],
              coeffs=[-0.4804+0.j,  0.3435+0.j, -0.4347+0.j,  0.5716+0.j,  0.091 +0.j,
  0.091 +0.j])


In [6]:
expanded_hamiltonian = expand_hamiltonian(hamiltonian, total_qubits=4)
print(expanded_hamiltonian)

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'YYII', 'XXII'],
              coeffs=[-0.4804+0.j,  0.3435+0.j, -0.4347+0.j,  0.5716+0.j,  0.091 +0.j,
  0.091 +0.j])
